In [1]:
import datetime as dt
import time
#import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Playing around with stuff

In [3]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
#[kM + k, kM+k + M] pour la k'eme tranche
indices = [index for k in range(3) for index in range(k*3 + k + 1, k*3 + k + 3 + 1)]
print(indices)

print([testarr[(k*3 + k + 1):(k*3 + k + 3 + 1)] for k in range(0, 3)])
print(testarr[indices])

[1, 2, 3, 5, 6, 7, 9, 10, 11]
[array([9, 9, 9]), array([9, 9, 9]), array([9, 9, 9])]
[9 9 9 9 9 9 9 9 9]


In [4]:
# ne pas oublier que le vecteur de parametres theta est dans R^{M^2 + M}
testL1norm = ut.L1NormPartialReg(shape=(1,3*(3+1)), S=indices, regLambda=1)

In [5]:
print(testL1norm(testarr))
print(testL1norm.prox(testarr, tau=1))

81.0
[1. 8. 8. 8. 2. 8. 8. 8. 3. 8. 8. 8.]


In [6]:
supportIndices = [0, 4]
injection = pxop.SubSample(10, supportIndices).T

x = np.ones(len(supportIndices))

print('injection:', injection(x))

injection: [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
indices = np.array([k + k*3 for k in range(3)], dtype="int32")
print(indices)
print(testarr[indices])

[0 4 8]
[1 2 3]


In [54]:
dim1 = 2
dim2 = 3

l1normdim1 = pxop.L1Norm(dim=dim1)
l2normdim2 = pxop.L2Norm(dim=dim2)

x1 = np.ones(dim1)
x2 = 2*np.ones(dim2)
bigx = np.hstack([x1, x2])

#sumOp = pxop.Sum(arg_shape=)
normOp = pxop.hstack([l1normdim1, l2normdim2])
print(normOp(bigx))
print(l1normdim1(x1) + l2normdim2(x2))

[5.46410162]
[5.46410162]


In [ ]:
toyM = 4
ez = np.array([k + k*toyM for k in range(toyM)], dtype="int32")
print(ez)

ez2 = [index for k in range(toyM) for index in range(k*toyM + k + 1, k*toyM + k + toyM + 1)]
print(ez2)

### Generation of the source (using a single realization)

In [26]:
path = "simulated_data/my_simulations/simu0.csv"
#path = "simulated_data/events0.csv"
beta = 5.0
hpL = HawkesLikelihood(path=path, beta=beta)

Computing A^1 out of 6, k1=410
Computing A^2 out of 6, k2=439
Computing A^3 out of 6, k3=408
Computing A^4 out of 6, k4=420
Computing A^5 out of 6, k5=442
Computing A^6 out of 6, k6=455


In [22]:
hpL.plot_realization()

In [27]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 0.0130
Lipschitz constant: 120.50002646738884


### Generation of the source (using multiple realizations)

In [3]:
totalProcesses = 20
beta = 5.0

paths = [f"simulated_data/events{i}.csv" for i in range(totalProcesses)]
#paths = [f"simulated_data/my_simulations/simu{i}.csv" for i in range(totalProcesses)]
allHPs = np.full(totalProcesses, None)

for i in range(totalProcesses):
    print(f"Initializing process {i+1}...")
    allHPs[i] = HawkesLikelihood(path=paths[i], beta=beta)
M = allHPs[0].M

Initializing process 1...
Computing A^1 out of 8, k1=100
Computing A^2 out of 8, k2=165
Computing A^3 out of 8, k3=244
Computing A^4 out of 8, k4=116
Computing A^5 out of 8, k5=255
Computing A^6 out of 8, k6=342
Computing A^7 out of 8, k7=226
Computing A^8 out of 8, k8=353
Initializing process 2...
Computing A^1 out of 8, k1=224
Computing A^2 out of 8, k2=137
Computing A^3 out of 8, k3=296
Computing A^4 out of 8, k4=169
Computing A^5 out of 8, k5=403
Computing A^6 out of 8, k6=370
Computing A^7 out of 8, k7=150
Computing A^8 out of 8, k8=620
Initializing process 3...
Computing A^1 out of 8, k1=176
Computing A^2 out of 8, k2=154
Computing A^3 out of 8, k3=277
Computing A^4 out of 8, k4=165
Computing A^5 out of 8, k5=390
Computing A^6 out of 8, k6=343
Computing A^7 out of 8, k7=171
Computing A^8 out of 8, k8=580
Initializing process 4...
Computing A^1 out of 8, k1=116
Computing A^2 out of 8, k2=145
Computing A^3 out of 8, k3=258
Computing A^4 out of 8, k4=165
Computing A^5 out of 8, k5=3

In [4]:
# Define big forward operator A: R^{M + M^2} -> R^{K1 + ... + Kn + totalProcesses*M}
bigA = pxop.vstack([allHPs[i].opA for i in range(totalProcesses)])

# Define big E operator E1(x_1, ..., x_K1) + E2(x_{K1+1}, ..., x_{K1+K2}) + ...
# E: R^{K1 + ... + Kn} -> R
bigE = pxop.hstack([allHPs[i].E for i in range(totalProcesses)])

# Define big -log(likelihood operator)
bigNegLogL = bigE * bigA

In [5]:
# print(bigE)
# print(bigNegLogL)
# x = 3*np.ones(bigA.shape[1])
# print(np.linalg.norm(bigA(x)))

In [34]:
start = time.time()
lip = bigA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)

Computation time of the Lipschitz constant: 0.4512
Lipschitz constant: 541.05278567072


In [7]:
allArrivals = np.array([])
for i in range(len(allHPs)):
    for j in range(M):
        allArrivals = np.hstack([allArrivals, allHPs[i].t[j]])

### Defining solvers

In [140]:
## Parameters of the solvers
# Base
remove = True  # originally True
min_iterations = 1
# Stop
eps = 1e-3  # relative error for objective function, original value 1e-4
tmax = 20.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.8  # original value 0.8, the lower the more atoms will HP select
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

# Parameter for reconstruction
#lambda_factor = 0.1
#lambda_ = lambda_factor * np.linalg.norm(bigA.T(allArrivals), np.infty)  # rule of thumb to define lambda
lambda_ = 4e1

In [141]:
# pfw = pyfwl.PFWLasso(
#     forwardOp=hpL.opA,
#     convexOp=hpL.E,
#     lambda_=lambda_,
#     M=hpL.M,
#     negLogL=hpL.negLogL,
#     ms_threshold=ms_threshold,
#     init_correction_prec=init_correction,
#     final_correction_prec=final_correction,
#     remove_positions=remove,
#     min_correction_steps=correction_steps,
#     show_progress=False,
# )
pfw = pyfwl.PFWLasso(
    forwardOp=bigA,
    convexOp=bigE,
    lambda_=lambda_,
    M=M,
    negLogL=bigNegLogL,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=True,
)

In [142]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [143]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
INFO -- [2023-12-28 20:22:38.694429] Iteration 0
	iteration: 0
	N_iter: 1.0
	RelError[objective_func]: 0.0
	duration: 0.01732
	AbsError[dcv]: inf
	Memorize[objective_func]: 21363.333530763626
new indices [71]
actual new indices: 1
new indices [21 41 51 53 71]
actual new indices: 4
new indices [ 1  5  8 11 12 17 21 23 26 31 33 35 37 41 42 44 51 53 61 62 68 69 71]
actual new indices: 18
new indices [ 1  5  8 17 21 23 26 31 33 35 37 39 41 44 49 50 51 53 61 62 68 69 71]
actual new indices: 3
new indices [ 1  5  8 21 23 26 35 41 44 51 53 61 68 69 71]
actual new indices: 0
new indices [ 1  8 21 26 35 41 44 51 53 61 68 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
new indices [21 26 41 44 51 53 69 71]
actual new indices: 0
	Atoms kept: 34/34
INFO -- [2023-12-28 20:22:40.32

### Evaluation of the solve

In [144]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: 45.8758
Final value of objective function:
	PFW : -159203.04


In [145]:
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]
#M = hpL.M
#print(data_p["x"][supp(data_p["x"])])
#print(data_p["x"])
for i in range(M):
    print(f"Process {i}:", data_p["x"][(i*(M + 1)):(i*(M + 1) + M + 1)])

# Arrange intensities in a M x M matrix
intensities = np.zeros((M, M))
mus = np.zeros((M,1))
for i in range(M):
    intensities[i, :] = data_p["x"][(i*(M + 1) + 1):(i*(M + 1) + M + 1)]
    mus[i] = data_p["x"][i*(M+1)]

Process 0: [3.45838424 1.02541873 0.         0.         0.         1.01973321
 0.         0.         1.12278458]
Process 1: [3.36374342 0.         0.81451968 0.85858934 0.         0.
 0.         0.         1.03813631]
Process 2: [3.84393909 0.         0.         1.94865975 0.         1.0066375
 0.         0.         1.30560255]
Process 3: [3.41255934 0.         0.         0.         0.87152912 0.
 0.89308987 0.         1.09664151]
Process 4: [3.82337667 0.92444795 0.         0.80760324 0.         2.09850444
 0.84973461 0.         1.33329541]
Process 5: [3.79376621 0.         0.         0.         0.76843447 0.85868538
 1.88512376 0.         1.72737683]
Process 6: [3.47085985 0.         0.         0.         0.         0.
 0.         1.01897495 1.0350875 ]
Process 7: [3.98340723 0.         0.         0.         0.         1.09046268
 1.66870713 0.         3.11292324]


In [146]:
# Visualize solution
fig, ax = plt.subplots(1, 2, figsize=(8,5))

# Plot alpha's
im = ax[1].imshow(intensities/(beta*beta), # la norme L1 est alpha/beta^2
                cmap='viridis', 
                interpolation='nearest', 
                extent=[1, M, M, 1])
fig.colorbar(im, ax=ax[1])  # Add a colorbar to show the scale
ax[1].set_title('L1 norms of the $h_{ij}$')

# Plot mu's
im = ax[0].imshow(mus,
                  cmap='viridis',
                  interpolation='nearest',
                  extent=[1, 2, M, 1])
fig.colorbar(im, ax=ax[0])  # Add a colorbar to show the scale
ax[0].set_title('Background intensities')

plt.show()

for i in range(M):
    print(f"mu{i+1}:", mus[i])

print(intensities/(beta*beta))

mu1: [3.45838424]
mu2: [3.36374342]
mu3: [3.84393909]
mu4: [3.41255934]
mu5: [3.82337667]
mu6: [3.79376621]
mu7: [3.47085985]
mu8: [3.98340723]
[[0.04101675 0.         0.         0.         0.04078933 0.
  0.         0.04491138]
 [0.         0.03258079 0.03434357 0.         0.         0.
  0.         0.04152545]
 [0.         0.         0.07794639 0.         0.0402655  0.
  0.         0.0522241 ]
 [0.         0.         0.         0.03486116 0.         0.03572359
  0.         0.04386566]
 [0.03697792 0.         0.03230413 0.         0.08394018 0.03398938
  0.         0.05333182]
 [0.         0.         0.         0.03073738 0.03434742 0.07540495
  0.         0.06909507]
 [0.         0.         0.         0.         0.         0.
  0.040759   0.0414035 ]
 [0.         0.         0.         0.         0.04361851 0.06674829
  0.         0.12451693]]


In [147]:
# Plot objective function value
mini = hist_p["Memorize[objective_func]"][-1]

plt.figure(figsize=(10, 8))
plt.suptitle("Objective function values")
plt.subplot(211)
plt.title("Log-scale")
plt.yscale("log")
plt.plot(
    hist_p["duration"],
    (hist_p["Memorize[objective_func]"] - mini),
    label="PFW",
    marker="+",
)

plt.legend()
plt.ylabel("OFV")

plt.subplot(212)
plt.plot(
    hist_p["duration"],
    (hist_p["Memorize[objective_func]"] - mini),
    label="PFW",
    marker="+",
)

plt.legend()
plt.ylabel("OFV")
plt.xlabel("Solving time")
plt.title("Linear scale")
plt.show()


In [148]:
# Plot DCV values
plt.figure(figsize=(10, 5))
plt.title("Dual certificate values")
plt.plot(hist_p["duration"], hist_p["AbsError[dcv]"], label="PFW", marker="+")
plt.legend()
#plt.hlines(1, 0, hist_v["duration"].max(), color='r', ls="--")
#plt.ylim(0, 1.02/lambda_factor)
plt.show()